##第一個TF Hello World（簡單版）

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
data = load_iris()
x_train, x_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.3, random_state=88)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(3)
])

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100)

In [ ]:
model.evaluate(x_test,  y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.2188 - accuracy: 0.9556


[0.2188202142715454, 0.9555555582046509]

In [ ]:
x_test[0]

array([5.4, 3. , 4.5, 1.5])

In [ ]:
y_test[0]

1

In [ ]:
pred = model.predict([[5.4, 3. , 4.5, 1.5]])
np.argmax(pred)

1

## 第一個TF Hello World(專家版)

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
data = load_iris()
x_train, x_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.3, random_state=88)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [ ]:
for iris_data, iris_labels in train_ds:
  print(iris_data, iris_labels)

In [ ]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(3)

  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)


model = MyModel()

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss') 
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(iris_data, iris_labels):
  with tf.GradientTape() as tape: #tf.GradientTape訓練模型
    predictions = model(iris_data, training=True) #確認訓練來源
    loss = loss_object(iris_labels, predictions) #建立Loss運算來源
  gradients = tape.gradient(loss, model.trainable_variables) #backward 反向傳播
  optimizer.apply_gradients(zip(gradients, model.trainable_variables)) #取得訓練的權重、偏移量優化

  train_loss(loss)
  train_accuracy(iris_labels, predictions)

In [ ]:
@tf.function
def test_step(iris_data, iris_labels):
  predictions = model(iris_data, training=False) #training=False 表示測試資料進行測試
  t_loss = loss_object(iris_labels, predictions) #建立Loss

  test_loss(t_loss)
  test_accuracy(iris_labels, predictions)

In [ ]:
EPOCHS = 50

for epoch in range(EPOCHS):
  #每一次epoch的時候做重整
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for iris_data, iris_labels in train_ds:
    train_step(iris_data, iris_labels)

  for test_iris_data, test_iris_labels in test_ds:
    test_step(test_iris_data, test_iris_labels)

  template = '迭代 {}, 訓練資料損失: {}, 訓練資料準確率: {}, 測試資料損失: {}, 測試資料準確率: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

迭代 1, 訓練資料損失: 1.0532121658325195, 訓練資料準確率: 29.52381134033203, 測試資料損失: 0.9019368886947632, 測試資料準確率: 42.22222137451172
迭代 2, 訓練資料損失: 0.9497863054275513, 訓練資料準確率: 34.28571701049805, 測試資料損失: 0.9381266832351685, 測試資料準確率: 33.333335876464844
迭代 3, 訓練資料損失: 0.9094387292861938, 訓練資料準確率: 41.904762268066406, 測試資料損失: 0.930955171585083, 測試資料準確率: 42.22222137451172
迭代 4, 訓練資料損失: 0.8707206845283508, 訓練資料準確率: 60.000003814697266, 測試資料損失: 0.8703829050064087, 測試資料準確率: 66.66667175292969
迭代 5, 訓練資料損失: 0.8288087844848633, 訓練資料準確率: 89.52381134033203, 測試資料損失: 0.7996152639389038, 測試資料準確率: 95.55555725097656
迭代 6, 訓練資料損失: 0.7959583401679993, 訓練資料準確率: 87.61904907226562, 測試資料損失: 0.7482044696807861, 測試資料準確率: 80.0
迭代 7, 訓練資料損失: 0.7708638310432434, 訓練資料準確率: 66.66667175292969, 測試資料損失: 0.7188720703125, 測試資料準確率: 80.0
迭代 8, 訓練資料損失: 0.7454959750175476, 訓練資料準確率: 72.38095092773438, 測試資料損失: 0.7050217390060425, 測試資料準確率: 84.44444274902344
迭代 9, 訓練資料損失: 0.719704806804657, 訓練資料準確率: 86.66666412353516, 測試資料損失: 0.6966571807861328, 測試